## Natural Language Processing with TensorFlow

In [13]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from helper_function import performance_metrics, compare_baseline_with_new_result
from tensorflow.keras import layers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import tensorflow_hub as hub

### 1. Data preparation

In [14]:
### Link folders

train_df = pd.read_csv("data/nlp/train.csv")
test_df = pd.read_csv("data/nlp/test.csv")

In [15]:
### Shuffle training dataframe

train_df_shuffled = train_df.sample(frac = 1, random_state = 42)

train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [16]:
### How many total sample?

print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [17]:
### Visualize random training samples

random_index = random.randint(0, len(train_df) - 5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index + 5].itertuples():
    _, text, target = row
    
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 1 (real disaster)
Text:
Please recover from the Typhoon. ????

---

Target: 1 (real disaster)
Text:
The real question is why is the tornado siren going off in Dyersburg?

---

Target: 1 (real disaster)
Text:
#Colorado #News Motorcyclist bicyclist injured in Denver collision on Broadway: At least two people were tak... http://t.co/2iAFPmqJeP

---

Target: 1 (real disaster)
Text:
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/cs8mYAunA4

---

Target: 0 (not real disaster)
Text:
USFS an acronym for United States Fire Service. http://t.co/8NAdrGr4xC

---



In [18]:
### Split training data into training and validation sets

train_sentence, val_sentence, train_label, val_label = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)

In [19]:
### Check the length of validation & training

print(f"Length of train sentence: {len(train_sentence)}, length of train label: {len(train_label)}")
print(f"Length of val sentence: {len(val_sentence)}, length of val label: {len(val_label)}")

length of train sentence: 6851, length of train label: 6851
length of val sentence: 762, length of val label: 762


In [20]:
# View first 10 training sentences and its label

train_sentence[:10], train_label[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

### 2. Converting text into numbers
Tokenization - word-level tokenization, character-level tokenization, sub-word tokenization <br>
Embeddings - own/ custom embedding, pre-learned embedding 

In [21]:
### Example of text vectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None)

In [22]:
### What is average number of tokens (words)?

print(f"The average of tokens is {round(sum(len(i.split()) for i in train_sentences) / len(train_sentences))}")

The average of tokens is 15


In [11]:
### Set text vectorization with custom variables
### Set max number of words to have in our vocabulary
### Max length for the sequences

max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
    output_mode = "int",
    output_sequence_length = max_length)

In [12]:
### Fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentence)

In [13]:
### Create sample sentence and tokenize it
### Check the output

sample_sentence = "There's a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [14]:
### Choose random sentence from the training dataset then tokenize it

random_sentence = random.choice(train_sentence)

print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")

text_vectorizer([random_sentence])

Original text:
#USGS M 1.4 - 4km E of Interlaken California: Time2015-08-06 00:52:25 UTC2015-08-05 17:52:25 -07:00 at ep... http://t.co/zqrcptLrUM #SM      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1452,  772, 1444, 6321, 1300,    6, 5352,   90, 1580, 6382, 1242,
        6360, 6377,   17, 1865]], dtype=int64)>

In [15]:
### Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### 3. Creating embedding layer

In [16]:
### Creating embedding layer
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             embeddings_initializer = "uniform",
                             input_length = max_length,
                             name = "embedding_layer") 

In [17]:
### Get a random sentence from training set
### Embed the random sentence

random_sentence = random.choice(train_sentence)

print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
http://t.co/9k1tqsAarM Suicide bomber kills 15 in Saudi security site mosque - Reuters http://t.co/Ev3nX9scx3      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00185242,  0.00072893, -0.0376213 , ...,  0.01914451,
          0.03461095, -0.02687767],
        [-0.0210801 , -0.0275659 , -0.0312755 , ...,  0.02100715,
         -0.01798378,  0.02837929],
        [ 0.02314771,  0.03732182,  0.00252379, ..., -0.02452084,
          0.02583529,  0.02132055],
        ...,
        [-0.01057015,  0.00640609, -0.02170891, ...,  0.01703012,
         -0.01875156, -0.02419301],
        [-0.01057015,  0.00640609, -0.02170891, ...,  0.01703012,
         -0.01875156, -0.02419301],
        [-0.01057015,  0.00640609, -0.02170891, ...,  0.01703012,
         -0.01875156, -0.02419301]]], dtype=float32)>

In [18]:
### Single token's embedding

sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-1.85241550e-03,  7.28927553e-04, -3.76212969e-02,  7.56398588e-03,
        2.06191428e-02, -2.68066768e-02, -8.39536265e-03,  1.41359605e-02,
       -3.15573588e-02,  3.24479975e-02, -3.53546739e-02, -3.13735381e-02,
        1.58467554e-02,  2.64225341e-02, -2.73786075e-02, -1.22269280e-02,
        7.41760805e-03, -4.99799363e-02,  3.61898579e-02, -1.32665150e-02,
       -3.72347981e-03,  6.82451576e-03, -3.90610695e-02,  4.50167097e-02,
        1.37189887e-02, -2.20862031e-02, -4.78941575e-02, -2.75693890e-02,
        1.08556636e-02, -2.44153496e-02,  4.02749069e-02, -4.79562990e-02,
       -4.42538261e-02,  4.76114191e-02, -3.84174809e-02,  1.25062130e-02,
       -9.49501991e-05, -3.03765889e-02,  1.82464235e-02,  1.42831728e-03,
       -2.20111366e-02, -4.74976562e-02,  3.74122709e-03,  1.74755789e-02,
        4.16668542e-02, -2.20945477e-02, -3.14154848e-02, -6.96597248e-03,
        3.63042206e-03,  2.40446068e-02, -4.19338234

### 4. Model building

Model 1 - Naive Bayes (baseline)

In [19]:
### Initialize constant variables
saved_dir_loc = "model_logs"

In [20]:
### Convert words to numbers using tfidf then model the text

first_model = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

first_model.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [21]:
### Check performance metrics
baseline_pred = first_model.predict(val_sentences)

baseline_result = performance_metrics(y_true = val_labels, y_pred = baseline_pred)
baseline_result

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

Model 2 - Simple dense model

In [22]:
### Create one dimensional strings inputs 
input_layer = layers.Input(shape = (1,), dtype = "string")

### Turn the input text into numbers
x = text_vectorizer(input_layer)

### Embedding the numerized numbers
x = embedding(x)

### Lower the dimensionality of the embedding
x = layers.GlobalAveragePooling1D()(x)

### Create the output layer for binary outputs 
output_layer = layers.Dense(1, activation = "sigmoid")(x)

### Construct the model
second_model = tf.keras.Model(input_layer, output_layer)

second_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

second_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_layer (Embedding  (None, 15, 128)           1280000   
 )                                                               
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                             

In [23]:
second_model_history = second_model.fit(train_sentences, train_labels, epochs = 5,
    validation_data = (val_sentences, val_labels), 
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = "simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230923-231642
Epoch 1/5
215/215 [==============================] - 6s 23ms/step - loss: 0.6103 - accuracy: 0.6913 - val_loss: 0.5368 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.4416 - accuracy: 0.8184 - val_loss: 0.4706 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 5s 22ms/step - loss: 0.3468 - accuracy: 0.8605 - val_loss: 0.4573 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 5s 21ms/step - loss: 0.2840 - accuracy: 0.8918 - val_loss: 0.4689 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2369 - accuracy: 0.9134 - val_loss: 0.4902 - val_accuracy: 0.7913


In [24]:
# Check validation results
second_model.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4902 - accuracy: 0.7913


[0.4901529550552368, 0.7913385629653931]

In [25]:
### Check embedding weights
embedding.weights

[<tf.Variable 'embedding_layer/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 0.0060296 ,  0.02295903, -0.04056848, ...,  0.03535229,
         -0.00143927, -0.00938205],
        [ 0.00577053,  0.0071948 , -0.04605361, ...,  0.02683257,
          0.04248062, -0.0252433 ],
        [ 0.00740892,  0.01506983, -0.02879825, ..., -0.00576946,
         -0.01646395,  0.0675428 ],
        ...,
        [-0.02877762, -0.03890579, -0.024528  , ..., -0.00092606,
         -0.03448058,  0.01577456],
        [-0.00516727,  0.06183866, -0.02421062, ...,  0.07621562,
          0.01492765,  0.00807954],
        [ 0.10762745,  0.0228096 , -0.10485741, ...,  0.07033586,
          0.08096483,  0.03530077]], dtype=float32)>]

In [26]:
### Other way to check embedding weights
embed_weights = second_model.get_layer("embedding_layer").get_weights()[0]

embed_weights.shape

(10000, 128)

In [27]:
# !tensorboard dev upload --logdir ./model_logs \
#   --name "First deep model on text data" \
#   --description "Trying a dense model with an embedding layer" \
#   --one_shot

In [28]:
# !tensorboard dev delete --experiment_id EXPERIMENT_ID_TO_DELETE

In [29]:
second_model_pred_prob = second_model.predict(val_sentences)

### Turn into single-dimension tensor of float
second_model_pred = tf.squeeze(tf.round(second_model_pred_prob))

second_model_result = performance_metrics(y_true = val_labels, y_pred = second_model_pred)

second_model_result

24/24 [==============================] - 0s 1ms/step


{'accuracy': 79.13385826771653,
 'precision': 0.7997458316766562,
 'recall': 0.7913385826771654,
 'f1': 0.7874035967950923}

In [30]:
compare_baseline_with_new_result(baseline_result = baseline_result, new_result = second_model_result)

Baseline accuracy: 79.27, New accuracy: 79.13, Difference: -0.13123359580052352
Baseline precision: 0.81, New precision: 0.80, Difference: -0.011393168744661009
Baseline recall: 0.79, New recall: 0.79, Difference: -0.001312335958005173
Baseline f1: 0.79, New f1: 0.79, Difference: 0.001184620990137386


Model 3 - LSTM

In [31]:
third_model_embedding = layers.Embedding(input_dim = max_vocab_length, output_dim = 128,
    embeddings_initializer = "uniform", input_length = max_length, name = "third_embedding_layer")

input_layer = layers.Input(shape = (1,), dtype = "string")

x = text_vectorizer(input_layer)
x = third_model_embedding(x)
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)

output_layer = layers.Dense(1, activation = "sigmoid")(x)

third_model = tf.keras.Model(input_layer, output_layer, name = "third_model_lstm")

third_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"])

third_model.summary()

Model: "third_model_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 third_embedding_layer (Emb  (None, 15, 128)           1280000   
 edding)                                                         
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 

In [32]:
third_model_history = third_model.fit(train_sentences, train_labels, epochs = 5, 
    validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "lstm")])

Saving TensorBoard log files to: model_logs/lstm/20230923-231707
Epoch 1/5
215/215 [==============================] - 9s 28ms/step - loss: 0.5098 - accuracy: 0.7505 - val_loss: 0.4613 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 6s 28ms/step - loss: 0.3171 - accuracy: 0.8716 - val_loss: 0.4738 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 6s 27ms/step - loss: 0.2189 - accuracy: 0.9199 - val_loss: 0.5906 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 6s 26ms/step - loss: 0.1534 - accuracy: 0.9442 - val_loss: 0.6188 - val_accuracy: 0.7454
Epoch 5/5
215/215 [==============================] - 6s 27ms/step - loss: 0.1125 - accuracy: 0.9591 - val_loss: 0.9497 - val_accuracy: 0.7598


In [33]:
# !tensorboard dev upload --logdir ./model_logs \
#   --name "First deep model on text data" \
#   --description "Trying a dense model with an embedding layer" \
#   --one_shot

In [34]:
third_model_pred_prob = third_model.predict(val_sentences)
third_model_pred = tf.squeeze(tf.round(third_model_pred_prob))

third_model_result = performance_metrics(y_true = val_labels, y_pred = third_model_pred)
third_model_result

24/24 [==============================] - 1s 6ms/step


{'accuracy': 75.98425196850394,
 'precision': 0.7637560697167074,
 'recall': 0.7598425196850394,
 'f1': 0.7563819709955472}

In [35]:
compare_baseline_with_new_result(baseline_result, third_model_result)

Baseline accuracy: 79.27, New accuracy: 75.98, Difference: -3.2808398950131163
Baseline precision: 0.81, New precision: 0.76, Difference: -0.04738293070460986
Baseline recall: 0.79, New recall: 0.76, Difference: -0.03280839895013121
Baseline f1: 0.79, New f1: 0.76, Difference: -0.029837004809407763


Model 4 - Bidirectonal RNN

In [36]:
forth_model_embedding = layers.Embedding(input_dim = max_vocab_length, output_dim = 128,
    embeddings_initializer = "uniform", input_length = max_length, name = "forth_embedding")

input_layer = layers.Input(shape = (1,), dtype = "string")

x = text_vectorizer(input_layer)
x = forth_model_embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)

output_layer = layers.Dense(1, activation = "sigmoid")(x)

forth_model = tf.keras.Model(input_layer, output_layer, name = "forth_model_bidirectional_rnn")

forth_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"])

forth_model.summary()

Model: "forth_model_bidirectional_rnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 forth_embedding (Embedding  (None, 15, 128)           1280000   
 )                                                               
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                     

In [37]:
forth_model_history = forth_model.fit(train_sentences, train_labels,
    epochs = 5, validation_data = (val_sentences, val_labels), 
    callbacks = [create_tensorboard_callback(SAVE_DIR, "bidirectional_rnn")])

Saving TensorBoard log files to: model_logs/bidirectional_rnn/20230923-231740
Epoch 1/5
215/215 [==============================] - 10s 30ms/step - loss: 0.5090 - accuracy: 0.7463 - val_loss: 0.4560 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.3101 - accuracy: 0.8740 - val_loss: 0.4825 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.2052 - accuracy: 0.9225 - val_loss: 0.6169 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1349 - accuracy: 0.9527 - val_loss: 0.6960 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 5s 25ms/step - loss: 0.0925 - accuracy: 0.9657 - val_loss: 0.9134 - val_accuracy: 0.7638


In [38]:
forth_model_pred_prob = forth_model.predict(val_sentences)
forth_model_pred = tf.squeeze(tf.round(forth_model_pred_prob))

forth_model_result = performance_metrics(val_labels, forth_model_pred)
forth_model_result

24/24 [==============================] - 1s 6ms/step


{'accuracy': 76.37795275590551,
 'precision': 0.7702681471235864,
 'recall': 0.7637795275590551,
 'f1': 0.7594175807340501}

In [39]:
compare_baseline_with_new_result(baseline_result, forth_model_result)

Baseline accuracy: 79.27, New accuracy: 76.38, Difference: -2.887139107611546
Baseline precision: 0.81, New precision: 0.77, Difference: -0.04087085329773088
Baseline recall: 0.79, New recall: 0.76, Difference: -0.02887139107611547
Baseline f1: 0.79, New f1: 0.76, Difference: -0.026801395070904843


Model 5 - One dimensional cnn

In [40]:
fifth_model_embedding = layers.Embedding(input_dim = max_vocab_length, output_dim = 128,
    embeddings_initializer = "uniform", input_length = max_length, name = "fifth_embedding")

input_layer = layers.Input(shape = (1,), dtype = "string")

x = text_vectorizer(input_layer)
x = fifth_model_embedding(x)
x = layers.Conv1D(filters = 32, kernel_size = 5, activation = "relu")(x)
x = layers.GlobalMaxPool1D()(x)

output_layer = layers.Dense(1, activation = "sigmoid")(x)

fifth_model = tf.keras.Model(input_layer, output_layer, name = "fifth_model_cnn_1d")

fifth_model.compile(loss = "binary_crossentropy", 
    optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

fifth_model.summary()

Model: "fifth_model_cnn_1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 fifth_embedding (Embedding  (None, 15, 128)           1280000   
 )                                                               
                                                                 
 conv1d (Conv1D)             (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d (Glob  (None, 32)                0         
 alMaxPooling1D)                                                 
                                                

In [41]:
fifth_model_history = fifth_model.fit(train_sentences, train_labels,
    epochs = 5, validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(SAVE_DIR, "cnn_1d")])

Saving TensorBoard log files to: model_logs/cnn_1d/20230923-231813
Epoch 1/5
215/215 [==============================] - 5s 21ms/step - loss: 0.5652 - accuracy: 0.7199 - val_loss: 0.4748 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3480 - accuracy: 0.8571 - val_loss: 0.4840 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2198 - accuracy: 0.9204 - val_loss: 0.5512 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1388 - accuracy: 0.9539 - val_loss: 0.6160 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0990 - accuracy: 0.9688 - val_loss: 0.6664 - val_accuracy: 0.7651


In [42]:
fifth_model_pred_prob = fifth_model.predict(val_sentences)
fifth_model_pred = tf.squeeze(tf.round(fifth_model_pred_prob))

fifth_model_result = performance_metrics(val_labels, fifth_model_pred)
fifth_model_result

24/24 [==============================] - 0s 3ms/step


{'accuracy': 76.50918635170603,
 'precision': 0.765325307490926,
 'recall': 0.7650918635170604,
 'f1': 0.7637474572934756}

In [43]:
compare_baseline_with_new_result(baseline_result, fifth_model_result)

Baseline accuracy: 79.27, New accuracy: 76.51, Difference: -2.7559055118110223
Baseline precision: 0.81, New precision: 0.77, Difference: -0.04581369293039128
Baseline recall: 0.79, New recall: 0.77, Difference: -0.027559055118110187
Baseline f1: 0.79, New f1: 0.76, Difference: -0.022471518511479327


Model 6 - Pretrained sentence encoder

In [59]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
    input_shape = [], dtype = tf.string, trainable = False, name = "USE")

sixth_model = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(64, activation = "relu"),
  layers.Dense(1, activation = "sigmoid")
], name = "sixth_model_use")

sixth_model.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

sixth_model.summary()

Model: "sixth_model_use"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_4 (Dense)             (None, 64)                32832     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [60]:
sixth_model_history = sixth_model.fit(train_sentences, train_labels,
    epochs = 5, validation_data = (val_sentences, val_labels), 
    callbacks=[create_tensorboard_callback(SAVE_DIR, "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20230923-232702
Epoch 1/5
215/215 [==============================] - 7s 20ms/step - loss: 0.5003 - accuracy: 0.7885 - val_loss: 0.4491 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.4146 - accuracy: 0.8133 - val_loss: 0.4411 - val_accuracy: 0.8097
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3999 - accuracy: 0.8228 - val_loss: 0.4313 - val_accuracy: 0.8150
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3911 - accuracy: 0.8288 - val_loss: 0.4332 - val_accuracy: 0.8150
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3851 - accuracy: 0.8305 - val_loss: 0.4248 - val_accuracy: 0.8150


In [61]:
sixth_model_pred_prob = sixth_model.predict(val_sentences)
sixth_model_pred = tf.squeeze(tf.round(sixth_model_pred_prob))

sixth_model_result = performance_metrics(val_labels, sixth_model_pred)
sixth_model_result

24/24 [==============================] - 1s 12ms/step


{'accuracy': 81.49606299212599,
 'precision': 0.8169586293569981,
 'recall': 0.8149606299212598,
 'f1': 0.8135344618830033}

In [62]:
compare_baseline_with_new_result(baseline_result, sixth_model_result)

Baseline accuracy: 79.27, New accuracy: 81.50, Difference: 2.230971128608928
Baseline precision: 0.81, New precision: 0.82, Difference: 0.005819628935680887
Baseline recall: 0.79, New recall: 0.81, Difference: 0.022309711286089273
Baseline f1: 0.79, New f1: 0.81, Difference: 0.027315486078048345
